In [1]:
%load_ext autoreload

In [23]:
import os, sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import tensorflow.keras as keras

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout

In [3]:
BASE_PATH = os.path.join(os.getcwd(), "..", "..")
DATASET_PATH = os.path.join(BASE_PATH, "datasets")
MODULES_PATH = os.path.join(BASE_PATH, "modules")
sys.path.append(MODULES_PATH)

In [4]:
TF_AL_PATH = os.path.join(BASE_PATH, "tf_al")
sys.path.append(TF_AL_PATH)

In [5]:
from tf_al import Config, ActiveLearningLoop, Dataset
from tf_al.utils import gen_seeds
from tf_al.wrapper import McDropout
from models import dnn_dense_dropout, disable_tf_logs, setup_growth

In [6]:
ABALONE_PATH = os.path.join(DATASET_PATH, "abalone")
columns = [
    "sex", 
    "length", 
    "diameter", 
    "height", 
    "whole_weight", 
    "shucked_weight", 
    "viscera_weight", 
    "shell_weight", 
    "rings"
]
data = pd.read_csv(os.path.join(ABALONE_PATH, "abalone.data"), sep=",")
data.columns = columns
encoder = LabelEncoder()
inputs = data[columns[:-1]]
inputs["sex"] = encoder.fit_transform(inputs["sex"].to_numpy())
inputs = inputs.to_numpy()
targets = data["rings"].to_numpy()
targets.astype(int)

/tmp/ipykernel_10458/303789098.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputs["sex"] = encoder.fit_transform(inputs["sex"].to_numpy())


array([ 7,  9, 10, ...,  9, 10, 12])

In [7]:
indices = []
for idx in range(len(targets)):
    
    if targets[idx] in [6, 7, 8, 9]:
        indices.append(idx)

In [8]:
inputs[indices].shape

(1907, 8)

In [9]:
np.any(np.isnan(inputs))

False

In [10]:
np.any(np.isnan(targets))

False

In [11]:
#seeds = gen_seeds(10)
# seeds = [3042034, 234234]
#np.random.seed(seeds[0])
#tf.random.set_seed(seeds[0])
#print(seeds)

In [12]:
num_input_features = len(columns[:-1])
num_classes = len(np.unique(targets))

In [13]:
print("Inputs: ", len(inputs))
print("Targets: ", len(targets))

Inputs:  4176
Targets:  4176


In [14]:
x_train, x_test, y_train, y_test = train_test_split(inputs, targets)
initial_pool_size = 100

In [15]:
dataset = Dataset(
    x_train, y_train,
    test=(x_test, y_test),
    init_size=500
)

In [16]:
x_train.shape

(3132, 8)

In [17]:
# Configure Tensorflow
disable_tf_logs()
setup_growth()

1 Physical GPU's,  1 Logical GPU's


In [63]:
batch_size = 10
sample_size = 25

#base_model = dnn_dense_dropout(num_input_features, 1, 3)
base_model = Sequential([
    Dense(8, activation="relu", input_shape=(8,)),
    Dropout(.25),
    Dense(128, activation="relu"),
    Dense(128, activation="relu"),
    Dense(num_classes, activation="softmax")
])

#optimizer = keras.optimizers.Adam(learning_rate=0.0001)
optimizer = "adam"
optimizer = "sgd"
loss = "sparse_categorical_crossentropy"
metrics = [keras.metrics.SparseCategoricalAccuracy()]

# --------------- ------
# MC Dropout Model
fit_params = {"epochs": 200, "batch_size": batch_size}
mc_config = Config(
    fit=fit_params,
    query={"sample_size": sample_size},
    eval={"batch_size": 900, "sample_size": sample_size}
)
mc_model = McDropout(base_model, config=mc_config)
mc_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [64]:
base_model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_42 (Dense)             (None, 8)                 72        
_________________________________________________________________
dropout_13 (Dropout)         (None, 8)                 0         
_________________________________________________________________
dense_43 (Dense)             (None, 128)               1152      
_________________________________________________________________
dense_44 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_45 (Dense)             (None, 28)                3612      
Total params: 21,348
Trainable params: 21,348
Non-trainable params: 0
_________________________________________________________________


In [65]:
base_model.fit(inputs, targets, epochs=200, batch_size=10)

Epoch 1/200
418/418 [==============================] - 1s 2ms/step - loss: nan - sparse_categorical_accuracy: 0.0797
Epoch 2/200
418/418 [==============================] - 1s 2ms/step - loss: nan - sparse_categorical_accuracy: 0.0000e+00
Epoch 3/200
418/418 [==============================] - 1s 2ms/step - loss: nan - sparse_categorical_accuracy: 0.0000e+00
Epoch 4/200
418/418 [==============================] - 1s 2ms/step - loss: nan - sparse_categorical_accuracy: 0.0000e+00
Epoch 5/200
418/418 [==============================] - 1s 2ms/step - loss: nan - sparse_categorical_accuracy: 0.0000e+00
Epoch 6/200
418/418 [==============================] - 1s 2ms/step - loss: nan - sparse_categorical_accuracy: 0.0000e+00
Epoch 7/200
418/418 [==============================] - 1s 2ms/step - loss: nan - sparse_categorical_accuracy: 0.0000e+00
Epoch 8/200
418/418 [==============================] - 1s 2ms/step - loss: nan - sparse_categorical_accuracy: 0.0000e+00
Epoch 9/200
418/418 [===============

KeyboardInterrupt: 

In [ ]:
acl = ActiveLearningLoop(
    mc_model,
    dataset,
    "random",
    step_size=10
)

In [21]:
data, o_targets = acl.pool.get_labeled_data()

In [22]:
base_model.fit(inputs, targets, epochs=200, batch_size=10)

Epoch 1/200
418/418 [==============================] - 1s 2ms/step - loss: 90.1782 - sparse_categorical_accuracy: 0.0000e+00
Epoch 2/200
418/418 [==============================] - 1s 2ms/step - loss: 90.1782 - sparse_categorical_accuracy: 0.0000e+00
Epoch 3/200
418/418 [==============================] - 1s 2ms/step - loss: 90.1782 - sparse_categorical_accuracy: 0.0000e+00
Epoch 4/200
418/418 [==============================] - 1s 2ms/step - loss: 90.1782 - sparse_categorical_accuracy: 0.0000e+00
Epoch 5/200
418/418 [==============================] - 1s 2ms/step - loss: 90.1782 - sparse_categorical_accuracy: 0.0000e+00
Epoch 6/200
418/418 [==============================] - 1s 2ms/step - loss: 90.1782 - sparse_categorical_accuracy: 0.0000e+00
Epoch 7/200
418/418 [==============================] - 1s 2ms/step - loss: 90.1782 - sparse_categorical_accuracy: 0.0000e+00
Epoch 8/200
418/418 [==============================] - 1s 2ms/step - loss: 90.1782 - sparse_categorical_accuracy: 0.0000e+00


KeyboardInterrupt: 

In [30]:
#mc_model.fit(data, targets)